# Подготовка данных

In [1]:
import requests
import pandas as pd

res = requests.get('http://10.32.1.107:5300/api/v1/physical_object_types')
physical_objects_types = pd.DataFrame(res.json()).set_index('physical_object_type_id', drop=True)
# physical_objects_types.to_excel('pot.xlsx')
physical_objects_types

,name,physical_object_function
physical_object_type_id,,
1,Рекреационная зона,"{'id': 3, 'name': 'Рекреационная зона'}"
2,Водный объект,"{'id': 4, 'name': 'Водный объект'}"
3,Зеленая зона,"{'id': 2, 'name': 'Зеленая зона'}"
4,Жилой дом,"{'id': 1, 'name': 'Здание'}"
5,Нежилое здание,"{'id': 1, 'name': 'Здание'}"
7,Площадка,"{'id': 5, 'name': 'Площадка'}"
8,Заправочная станция,"{'id': 9, 'name': 'Заправочная станция'}"
9,Памятник,"{'id': 10, 'name': 'Достопримечательность'}"
10,Остановка наземного транспорта,"{'id': 8, 'name': 'Остановка'}"


In [2]:
from enum import Enum
class EngineeringObject(Enum):
    POWER_SUPPLY = 'Энергоснабжение'
    HEAT_SUPPLY = 'Теплоснабжение'
    GAS_SUPPLY = 'Газоснабжение'
    WATER_SUPPLY = 'Водоснабжение'
    WATER_DRAINAGE = 'Водоотведение'

In [3]:
ENG_OBJ = {
    EngineeringObject.POWER_SUPPLY: [14, 20, 21, 33, 34, 35],  # Электрические подстанции, ЛЭП, электростанции и генераторы
    EngineeringObject.HEAT_SUPPLY: [41],                       # Котельная
    EngineeringObject.GAS_SUPPLY: [13, 18],                    # Магистральный газопровод
    EngineeringObject.WATER_SUPPLY: [27, 38, 40, 42],          # Сети водоснабжения, водонапорные башни, водозаборные и насосные станции
    EngineeringObject.WATER_DRAINAGE: [24, 37, 39]             # Сети водоотведения, сооружения для очистки воды, водоочистные сооружения
}



In [4]:
import requests
import geopandas as gpd
import shapely
import json
import pandas as pd

URBAN_API = 'http://10.32.1.107:5300'
PAGE_SIZE = 10_000

def _get_physical_objects(region_id : int, pot_id : int, page : int, page_size : int = PAGE_SIZE):
  res = requests.get(f'{URBAN_API}/api/v1/territory/{region_id}/physical_objects_with_geometry', {
    'physical_object_type_id': pot_id,
    'page': page,
    'page_size': page_size,
  })
  return res.json()

def get_physical_objects(region_id : int, pot_id : int):
  page = 1
  results = []
  while True:
    res_json = _get_physical_objects(region_id, pot_id, page, page_size=PAGE_SIZE)
    results.extend(res_json['results'])
    if res_json['next'] is None:
      break
    page += 1
  
  # Фильтрация объектов с наличием geometry
  results_with_geometry = [result for result in results if 'geometry' in result and result['geometry'] is not None]
  
  # Если нет объектов с геометрией, возвращаем пустой GeoDataFrame
  if not results_with_geometry:
    return gpd.GeoDataFrame(columns=['geometry'])
  
  # Конвертация geometry
  for result in results_with_geometry:
    g = result['geometry']
    result['geometry'] = shapely.from_geojson(json.dumps(g))
  
  return gpd.GeoDataFrame(results_with_geometry).set_geometry('geometry')

def fetch_required_objects(region_id : int, pot_ids : list[int]):
  gdfs = [get_physical_objects(region_id, pot_id) for pot_id in pot_ids]
  return pd.concat(gdfs).set_geometry('geometry').set_crs(4326)

In [5]:
# def fetch_required_objects(region_id : int, pot_ids : list[int]):
#   gdfs = [get_physical_objects(region_id, pot_id) for pot_id in pot_ids]
#   return pd.concat(gdfs).set_geometry('geometry').set_crs(4326)

# gdf = fetch_required_objects(1, [4])
# gdf

In [6]:
gdfs = {}
for eng_obj, ind_id in ENG_OBJ.items():
  if len(ind_id) > 0:
    gdf = fetch_required_objects(1, ind_id)
    gdfs[eng_obj] = gdf
gdfs

{<EngineeringObject.POWER_SUPPLY: 'Энергоснабжение'>:                      geometry  physical_object_id  \
 0   POINT (30.50513 59.84176)            759813.0   
 1   POINT (30.50316 59.84188)            759823.0   
 2    POINT (30.42919 60.0376)            759836.0   
 3   POINT (30.49322 59.89584)            759924.0   
 4   POINT (30.26719 60.11068)            759957.0   
 5   POINT (29.84225 59.87058)            759959.0   
 6   POINT (30.07734 59.77243)            759960.0   
 7   POINT (30.47855 59.90406)            759965.0   
 8   POINT (30.09877 59.78206)            759969.0   
 9     POINT (30.49359 59.939)            759973.0   
 10  POINT (30.48904 59.94282)            759984.0   
 11  POINT (30.48905 59.94284)            759990.0   
 12  POINT (30.48398 59.94375)            760001.0   
 
                                  physical_object_type  \
 0   {'physical_object_type_id': 35, 'name': 'Атомн...   
 1   {'physical_object_type_id': 35, 'name': 'Атомн...   
 2   {'physical

In [7]:
def get_engineering_gdf(data_dict: dict) -> gpd.GeoDataFrame:
    combined_gdf = gpd.GeoDataFrame(columns=['type', 'geometry'], crs="EPSG:4326")

    for eng_obj, gdf in data_dict.items():
        gdf = gdf.copy()  
        gdf['type'] = eng_obj.value 
        combined_gdf = pd.concat([combined_gdf, gdf], ignore_index=True)

    combined_gdf = gpd.GeoDataFrame(combined_gdf, geometry='geometry', crs="EPSG:4326")
    return combined_gdf


In [8]:
combined_gdf = get_engineering_gdf(gdfs)
combined_gdf

,type,geometry,physical_object_id,physical_object_type,name,address,osm_id,properties,centre_point,created_at,updated_at
0,Энергоснабжение,POINT (30.50513 59.84176),759813.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.5051266, ...",2024-10-14T10:07:08.813499Z,2024-10-14T10:07:08.813499Z
1,Энергоснабжение,POINT (30.50316 59.84188),759823.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.5031632, ...",2024-10-14T10:07:11.617739Z,2024-10-14T10:07:11.617739Z
2,Энергоснабжение,POINT (30.42919 60.0376),759836.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.4291869, ...",2024-10-14T10:07:14.388877Z,2024-10-14T10:07:14.388877Z
3,Энергоснабжение,POINT (30.49322 59.89584),759924.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.4932235, ...",2024-10-14T10:07:55.918200Z,2024-10-14T10:07:55.918200Z
4,Энергоснабжение,POINT (30.26719 60.11068),759957.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.2671947, ...",2024-10-14T10:08:03.769546Z,2024-10-14T10:08:03.769546Z
5,Энергоснабжение,POINT (29.84225 59.87058),759959.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [29.8422486, ...",2024-10-14T10:08:04.246719Z,2024-10-14T10:08:04.246719Z
6,Энергоснабжение,POINT (30.07734 59.77243),759960.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.077335, 5...",2024-10-14T10:08:04.257549Z,2024-10-14T10:08:04.257549Z
7,Энергоснабжение,POINT (30.47855 59.90406),759965.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.4785509, ...",2024-10-14T10:08:06.249964Z,2024-10-14T10:08:06.249964Z
8,Энергоснабжение,POINT (30.09877 59.78206),759969.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.0987739, ...",2024-10-14T10:08:07.013439Z,2024-10-14T10:08:07.013439Z
9,Энергоснабжение,POINT (30.49359 59.939),759973.0,"{'physical_object_type_id': 35, 'name': 'Атомн...",(Physical object for (unnamed)),None,None,{},"{'type': 'Point', 'coordinates': [30.4935943, ...",2024-10-14T10:08:08.587837Z,2024-10-14T10:08:08.587837Z


In [9]:
combined_gdf.explore()

In [10]:
spb_hex = gpd.read_file('data/spb_hex.geojson')
spb_hex

,geometry
0,"POLYGON ((30.12453 60.01677, 30.14338 60.01106..."
1,"POLYGON ((30.23961 60.09269, 30.2585 60.08696,..."
2,"POLYGON ((30.40522 59.85712, 30.42398 59.85134..."
3,"POLYGON ((30.19969 59.98166, 30.21852 59.97593..."
4,"POLYGON ((30.3715 60.04029, 30.39036 60.03454,..."
...,...
611,"POLYGON ((30.3081 59.72653, 30.3268 59.72075, ..."
612,"POLYGON ((30.26976 59.689, 30.28843 59.68323, ..."
613,"POLYGON ((30.27763 60.09348, 30.29652 60.08775..."
614,"POLYGON ((30.06399 59.73999, 30.08269 59.73426..."


# РАБОТАЕМ

In [11]:
from popframe.method.engineer import InfrastructureAnalyzer

analyzer = InfrastructureAnalyzer(combined_gdf, spb_hex)
results = analyzer.get_results()
results

,score,types_in_radius,geometry
0,1,[Энергоснабжение],"POLYGON ((30.12453 60.01677, 30.14338 60.01106..."
1,1,[Энергоснабжение],"POLYGON ((30.23961 60.09269, 30.2585 60.08696,..."
2,1,[Энергоснабжение],"POLYGON ((30.40522 59.85712, 30.42398 59.85134..."
3,1,[Энергоснабжение],"POLYGON ((30.19969 59.98166, 30.21852 59.97593..."
4,1,[Энергоснабжение],"POLYGON ((30.3715 60.04029, 30.39036 60.03454,..."
...,...,...,...
611,1,[Энергоснабжение],"POLYGON ((30.3081 59.72653, 30.3268 59.72075, ..."
612,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.26976 59.689, 30.28843 59.68323, ..."
613,1,[Энергоснабжение],"POLYGON ((30.27763 60.09348, 30.29652 60.08775..."
614,1,[Энергоснабжение],"POLYGON ((30.06399 59.73999, 30.08269 59.73426..."


In [12]:
results[results['score']>1]

,score,types_in_radius,geometry
75,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.1021 59.77753, 30.12082 59.7718, ..."
200,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.25131 59.70704, 30.27 59.70127, 3..."
212,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.12118 59.79631, 30.13991 59.79057..."
429,2,"[Энергоснабжение, Водоотведение]","POLYGON ((30.02734 59.81268, 30.04608 59.80696..."
527,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.13972 59.77832, 30.15844 59.77258..."
532,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.47995 59.82172, 30.49869 59.81593..."
537,2,"[Энергоснабжение, Водоотведение]","POLYGON ((30.06498 59.81349, 30.08372 59.80777..."
612,2,"[Водоснабжение, Энергоснабжение]","POLYGON ((30.26976 59.689, 30.28843 59.68323, ..."
